## Install AutoTrain

In [ ]:
!pip install -Uqq autotrain-advanced

## Export your Hugging Face credentials

In [ ]:
!export HF_USERNAME=your_hugging_face_username
!export HF_TOKEN=your_hugging_face_write_token

## Run AutoTrain

In [ ]:
!git clone https://github.com/dnth/vl-hf-annotation-converter
!cd vl-hf-annotation-converter && autotrain --config configs/detr-resnet-50.yml

In [ ]:
!cd vl-hf-annotation-converter && python inference.py